In [26]:
import os
import logging
from pathlib import Path

import numpy as np
import pandas as pd
import librosa
import torch
import torch.nn.functional as F
from tqdm.auto import tqdm
import sys
from joblib import Parallel, delayed
logging.basicConfig(level=logging.ERROR)


import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
# 
class CFG:
    def __init__(self, mode="train", kaggle_notebook=False, debug=False):
        assert mode in ["train", "inference"], "mode must be 'train' or 'inference'"
        self.mode = mode
        self.KAGGLE_NOTEBOOK = kaggle_notebook
        self.debug = debug

        # ===== Path Settings =====
        if self.KAGGLE_NOTEBOOK:
            self.OUTPUT_DIR = ''
            self.train_datadir = '/kaggle/input/birdclef-2025/train_audio'
            self.train_csv = '/kaggle/input/birdclef-2025/train.csv'
            self.test_soundscapes = '/kaggle/input/birdclef-2025/test_soundscapes'
            self.submission_csv = '/kaggle/input/birdclef-2025/sample_submission.csv'
            self.taxonomy_csv = '/kaggle/input/birdclef-2025/taxonomy.csv'
            self.spectrogram_npy = '/kaggle/input/birdclef25-mel-spectrograms/birdclef2025_melspec_5sec_256_256.npy'
            
            # kaggle notebookならここを変更
            self.model_path = "/kaggle/input/birdclef-2025-baseline-fold0-0404"
            
            self.device = "cpu"
            self.batch_size = 8
            self.n_jobs = 3
            
        else:
            self.OUTPUT_DIR = '../data/result/'
            self.train_datadir = '../data/raw/train_audio/'
            self.train_csv = '../data/raw/train.csv'
            self.test_soundscapes = '../data/raw/test_soundscapes_small/'
            self.submission_csv = '../data/raw/sample_submission.csv'
            self.taxonomy_csv = '../data/raw/taxonomy.csv'
            self.spectrogram_npy = '../data/processed/mel-spec_0329/birdclef2025_melspec_5sec_256_256.npy'
            self.MODELS_DIR = "../models/"
            
            # ローカルならここを変更
            self.model_path =  "../models/fold0_cleaned_10sec0423_vino/"
            
            self.device = "cuda" if torch.cuda.is_available() else "cpu"
            self.batch_size = 32
            self.n_jobs = 3

        # ===== Model Settings =====
        self.model_name = 'efficientnet_b0'
        self.pretrained = True if mode == "train" else False
        self.in_channels = 1

        # ===== Audio Settings =====
        self.FS = 32000
        self.TARGET_SHAPE = (148, 640)
        self.N_FFT = 1024
        self.HOP_LENGTH = 64
        self.N_MELS = 148
        self.FMIN = 20
        self.FMAX = 16000
        
        self.seed = 42
        
        self.WINDOW_SIZE = 10 # 切り取る音声の秒数
        
        # smoothingの係数
        self.smooth_center_weight = 0.6
        self.smooth_neighbor_weight = 0.2
        
        # ===== Inference Mode =====
        if mode == "inference":
            self.use_tta = False
            self.tta_count = 3
            self.threshold = 0.5

            self.use_specific_folds = False
            self.folds = [0, 1, 2, 3, 4]  # Used only if use_specific_folds is True

            self.debug_count = 3

            
            
    def update_debug_settings(self):
        if self.debug:
            self.epochs = 2
            self.selected_folds = [0]

In [28]:
cfg = CFG(mode='inference', kaggle_notebook=False)

if cfg.KAGGLE_NOTEBOOK:
    !pip install -U openvino-telemetry  --no-index --find-links /kaggle/input/pip-hub
    !pip install -U openvino  --no-index --find-links /kaggle/input/pip-hub
    sys.path.append("/kaggle/input/birdclef-2025-libs/")
    
from openvino.runtime import Core
from module import models_lib, utils_lib, preprocess_lib, inference_lib

# Set seed
utils_lib.set_seed(cfg.seed)

In [29]:
print(f"Using device: {cfg.device}")
print(f"Loading taxonomy data...")
taxonomy_df = pd.read_csv(cfg.taxonomy_csv)
species_ids = taxonomy_df['primary_label'].tolist()
num_classes = len(species_ids)
print(f"Number of classes: {num_classes}")

Using device: cuda
Loading taxonomy data...
Number of classes: 206


In [30]:
# mel変換
def process_audio_file(audio_path, cfg):
    """
    row_id = 5〜60（12個）に対する10秒ウィンドウの start/end を明示的に定義して処理。
    """
    dataset = []
    soundscape_id = Path(audio_path).stem
    audio_data, _ = librosa.load(audio_path, sr=cfg.FS)
    fs = cfg.FS

    # === 明示的なウィンドウ定義 ===　←頭いい
    segments = [
        (5, 0.0, 10.0),
        (10, 2.5, 12.5),
        (15, 7.5, 17.5),
        (20, 12.5, 22.5),
        (25, 17.5, 27.5),
        (30, 22.5, 32.5),
        (35, 27.5, 37.5),
        (40, 32.5, 42.5),
        (45, 37.5, 47.5),
        (50, 42.5, 52.5),
        (55, 47.5, 57.5),
        (60, 50.0, 60.0),
    ]

    for row_id_time, start_sec, end_sec in segments:
        start_idx = int(start_sec * fs)
        end_idx = int(end_sec * fs)
        segment = audio_data[start_idx:end_idx]

        mel_spec = preprocess_lib.process_audio_segment(segment, cfg)
        row_id = f"{soundscape_id}_{row_id_time}"
        dataset.append({
            "row_id": row_id,
            "mel_spec": mel_spec
        })

    return dataset





# 並列化してmelspecを生成
def generate_melspec_dataset(cfg):
    test_dir = Path(cfg.test_soundscapes)
    if not test_dir.exists():
        print(f"Test directory {test_dir} does not exist.")
        return []

    test_files = list(test_dir.glob('*.ogg'))
    if len(test_files) == 0:
        print("No test audio files found.")
        return []

    if cfg.debug:
        print(f"Debug mode enabled, using only {cfg.debug_count} files")
        test_files = test_files[:cfg.debug_count]

    results = Parallel(n_jobs=cfg.n_jobs)(
        delayed(process_audio_file)(path, cfg) for path in tqdm(test_files, desc="Parallel melspec gen")
    )
    dataset = [item for sublist in results for item in sublist]
    return dataset

In [31]:

# openvinoモデルの読み込み
def load_openvino_models(vino_dir, cfg):
    models = []
    vino_dir = Path(vino_dir)

    if cfg.use_specific_folds:
        fold_ids = cfg.folds
        xml_files = [vino_dir / f"model_fold{f}.xml" for f in fold_ids]
    else:
        xml_files = sorted(vino_dir.glob("model_fold*.xml"))

    for xml_path in xml_files:
        bin_path = xml_path.with_suffix(".bin")

        if not xml_path.exists() or not bin_path.exists():
            print(f"⚠️ Warning: Missing files for {xml_path.stem}")
            continue

        core = Core()
        model_ir = core.read_model(xml_path)
        compiled_model = core.compile_model(model_ir, device_name="CPU")
        models.append(compiled_model)

        # 🔍 モデルのファイル名（fold情報）をログに出す
        print(f"✅ Loaded model: {xml_path.name}")

    print(f"🎉 Total {len(models)} OpenVINO model(s) loaded from {vino_dir}")
    return models

# openvinoモデルによる推論
def run_inference_openvino(dataset, models_ir, cfg, species_ids):
    row_ids = []
    all_preds = []

    for i in range(0, len(dataset), cfg.batch_size):
        batch = dataset[i:i+cfg.batch_size]

        mel_list = [item["mel_spec"] for item in batch]

        # === shape 修正 ===
        input_tensor = np.stack(mel_list).astype(np.float32)   # (B, W, H)
        input_tensor = input_tensor.transpose(0, 2, 1)          # (B, H, W)
        input_tensor = np.expand_dims(input_tensor, axis=1)    # (B, 1, H, W)

        preds_per_model = []
        for model in models_ir:
            input_layer = model.input(0)
            output_layer = model.output(0)
            result = model([input_tensor])[output_layer]
            probs = 1 / (1 + np.exp(-result))  # sigmoid
            preds_per_model.append(probs)

        avg_preds = np.mean(preds_per_model, axis=0)
        all_preds.append(avg_preds)
        row_ids.extend([item["row_id"] for item in batch])

    predictions = np.concatenate(all_preds, axis=0)
    return row_ids, predictions

In [32]:
# 予測値をsmoothingする．
def smooth_submission_df(submission_df, cfg):
    """
    Smooth predictions in a submission DataFrame and return the modified DataFrame.

    Each row's prediction is averaged with its neighbors using weights from cfg.

    :param submission_df: DataFrame containing submission data (row_id + prediction columns)
    :param cfg: Configuration object containing smoothing weights
    :return: Smoothed submission DataFrame
    """
    print("Smoothing submission predictions...")
    sub = submission_df.copy()
    cols = sub.columns[1:]
    groups = sub['row_id'].astype(str).str.rsplit('_', n=1).str[0].values
    unique_groups = np.unique(groups)

    for group in unique_groups:
        idx = np.where(groups == group)[0]
        sub_group = sub.iloc[idx].copy()
        predictions = sub_group[cols].values
        new_predictions = predictions.copy()

        if predictions.shape[0] > 1:
            w_c = cfg.smooth_center_weight
            w_n = cfg.smooth_neighbor_weight
            new_predictions[0] = predictions[0] * (1 - w_n) + predictions[1] * w_n
            new_predictions[-1] = predictions[-1] * (1 - w_n) + predictions[-2] * w_n
            for i in range(1, predictions.shape[0] - 1):
                new_predictions[i] = (
                    predictions[i - 1] * w_n +
                    predictions[i] * w_c +
                    predictions[i + 1] * w_n
                )
        sub.iloc[idx, 1:] = new_predictions

    return sub

In [33]:
print("Generating dataset...")
dataset = generate_melspec_dataset(cfg)

print("Loading OpenVINO models...")
vino_dir = Path(cfg.model_path).with_name(Path(cfg.model_path).name)
models_ir = load_openvino_models(vino_dir, cfg)

if not models_ir:
    raise RuntimeError("No OpenVINO models found.")

print("Running OpenVINO inference...")
if len(dataset) > 0:
    row_ids, predictions = run_inference_openvino(dataset, models_ir, cfg, species_ids)
else:
    print("No test data available, generating empty submission.")
    row_ids = []
    predictions = []

# smoothing前の予測値を保存
submission_df = utils_lib.create_submission(row_ids, predictions, species_ids, cfg)
submission_path = os.path.join(cfg.OUTPUT_DIR, 'submission_before_smoothing.csv')
submission_df.to_csv(submission_path, index=False)

print(f"Submission saved to {submission_path}")


Generating dataset...


Parallel melspec gen:   0%|          | 0/71 [00:00<?, ?it/s]

Loading OpenVINO models...
✅ Loaded model: model_fold0.xml
🎉 Total 1 OpenVINO model(s) loaded from ../models/fold0_cleaned_10sec0423_vino
Running OpenVINO inference...
Creating submission dataframe...
Submission saved to ../data/result/submission_before_smoothing.csv


In [34]:
# smoothingして再度保存．
smoothed_df = smooth_submission_df(submission_df, cfg)
smoothed_submission_path = os.path.join(cfg.OUTPUT_DIR, 'submission.csv')
smoothed_df.to_csv(smoothed_submission_path, index=False)
print(f"Smoothed submission saved to {smoothed_submission_path}")

Smoothing submission predictions...
Smoothed submission saved to ../data/result/submission.csv


In [35]:
# 提出用ファイルを読み込む
submission = pd.read_csv(os.path.join(cfg.OUTPUT_DIR, 'submission.csv'))
submission.head(12)

,row_id,1139490,1192948,1194042,126247,1346504,134933,135045,1462711,1462737,...,yebfly1,yebsee1,yecspi2,yectyr1,yehbla2,yehcar1,yelori1,yeofly1,yercac1,ywcpar
0,H02_20230420_074000_5,0.002838,0.000612,0.003572,0.006501,0.002966,0.000358,0.003202,0.002460,0.001898,...,0.024276,0.032099,0.001829,0.012634,0.003107,0.024436,0.007257,0.007632,0.017838,0.002737
1,H02_20230420_074000_10,0.003339,0.000767,0.004278,0.008478,0.003880,0.000463,0.004309,0.003219,0.002847,...,0.029016,0.044144,0.002937,0.016421,0.004675,0.039175,0.008696,0.007723,0.029685,0.003981
2,H02_20230420_074000_15,0.006270,0.001655,0.007749,0.013290,0.007315,0.001061,0.008330,0.005879,0.005246,...,0.052004,0.067186,0.005122,0.026516,0.007750,0.047516,0.015025,0.014737,0.051728,0.006969
3,H02_20230420_074000_20,0.008450,0.002368,0.010168,0.016450,0.009969,0.001593,0.011802,0.007715,0.006719,...,0.067973,0.072437,0.005684,0.033886,0.009674,0.041778,0.017866,0.022489,0.054851,0.008784
4,H02_20230420_074000_25,0.004838,0.001368,0.005434,0.015068,0.005313,0.001095,0.006183,0.004271,0.003833,...,0.040782,0.048662,0.003032,0.022294,0.006173,0.024644,0.010409,0.014700,0.031442,0.005981
5,H02_20230420_074000_30,0.003310,0.000762,0.003115,0.008917,0.003270,0.000622,0.004031,0.002823,0.002524,...,0.031623,0.034101,0.002279,0.016619,0.003521,0.021623,0.006417,0.009237,0.025730,0.004032
6,H02_20230420_074000_35,0.003771,0.000946,0.004170,0.008243,0.003937,0.000599,0.005761,0.003611,0.002961,...,0.030994,0.047620,0.004178,0.018210,0.005164,0.029216,0.006898,0.007813,0.026520,0.003266
7,H02_20230420_074000_40,0.003970,0.001006,0.005078,0.008685,0.004340,0.000572,0.005538,0.003802,0.002990,...,0.032348,0.051369,0.003989,0.017968,0.005559,0.032201,0.008799,0.007829,0.025286,0.003277
8,H02_20230420_074000_45,0.004243,0.000937,0.005131,0.010305,0.004827,0.000598,0.004821,0.003815,0.003338,...,0.034345,0.044418,0.003220,0.015660,0.004215,0.032407,0.009091,0.007431,0.024762,0.003270
9,H02_20230420_074000_50,0.004831,0.001111,0.005391,0.013327,0.005064,0.000798,0.005812,0.004293,0.003936,...,0.035626,0.047756,0.003387,0.019120,0.004662,0.032377,0.010061,0.010736,0.028901,0.004491


In [36]:
submission = pd.read_csv(os.path.join(cfg.OUTPUT_DIR, 'submission_before_smoothing.csv'))
submission.head()

,row_id,1139490,1192948,1194042,126247,1346504,134933,135045,1462711,1462737,...,yebfly1,yebsee1,yecspi2,yectyr1,yehbla2,yehcar1,yelori1,yeofly1,yercac1,ywcpar
0,H02_20230420_074000_5,0.002890,0.000626,0.003594,0.006290,0.002890,0.000368,0.003075,0.002397,0.001755,...,0.024423,0.030215,0.001648,0.012054,0.002801,0.020023,0.007346,0.008062,0.015906,0.002551
1,H02_20230420_074000_10,0.002632,0.000553,0.003483,0.007346,0.003273,0.000315,0.003707,0.002715,0.002473,...,0.023689,0.039639,0.002551,0.014957,0.004332,0.042088,0.006904,0.005911,0.025565,0.003483
2,H02_20230420_074000_15,0.005911,0.001549,0.007346,0.014064,0.006693,0.001001,0.007346,0.005555,0.005060,...,0.049589,0.071591,0.005385,0.025179,0.007577,0.049589,0.015425,0.012821,0.055823,0.006904
3,H02_20230420_074000_20,0.010987,0.003075,0.013223,0.016915,0.013223,0.001988,0.015906,0.010014,0.008577,...,0.087564,0.081520,0.006904,0.042088,0.011687,0.046725,0.021948,0.029312,0.065605,0.010653
4,H02_20230420_074000_25,0.003377,0.001065,0.003824,0.017442,0.003483,0.001001,0.003945,0.002981,0.002801,...,0.027585,0.046034,0.002323,0.017986,0.005730,0.019124,0.008062,0.011687,0.021615,0.005060


In [37]:
submission

,row_id,1139490,1192948,1194042,126247,1346504,134933,135045,1462711,1462737,...,yebfly1,yebsee1,yecspi2,yectyr1,yehbla2,yehcar1,yelori1,yeofly1,yercac1,ywcpar
0,H02_20230420_074000_5,0.002890,0.000626,0.003594,0.006290,0.002890,0.000368,0.003075,0.002397,0.001755,...,0.024423,0.030215,0.001648,0.012054,0.002801,0.020023,0.007346,0.008062,0.015906,0.002551
1,H02_20230420_074000_10,0.002632,0.000553,0.003483,0.007346,0.003273,0.000315,0.003707,0.002715,0.002473,...,0.023689,0.039639,0.002551,0.014957,0.004332,0.042088,0.006904,0.005911,0.025565,0.003483
2,H02_20230420_074000_15,0.005911,0.001549,0.007346,0.014064,0.006693,0.001001,0.007346,0.005555,0.005060,...,0.049589,0.071591,0.005385,0.025179,0.007577,0.049589,0.015425,0.012821,0.055823,0.006904
3,H02_20230420_074000_20,0.010987,0.003075,0.013223,0.016915,0.013223,0.001988,0.015906,0.010014,0.008577,...,0.087564,0.081520,0.006904,0.042088,0.011687,0.046725,0.021948,0.029312,0.065605,0.010653
4,H02_20230420_074000_25,0.003377,0.001065,0.003824,0.017442,0.003483,0.001001,0.003945,0.002981,0.002801,...,0.027585,0.046034,0.002323,0.017986,0.005730,0.019124,0.008062,0.011687,0.021615,0.005060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
847,H02_20230502_080500_40,0.004199,0.001170,0.005385,0.005555,0.004468,0.000380,0.009708,0.004468,0.003173,...,0.025179,0.051845,0.012821,0.029312,0.008062,0.035145,0.007346,0.004610,0.048137,0.001810
848,H02_20230502_080500_45,0.000688,0.000779,0.001134,0.000245,0.001245,0.000169,0.002473,0.000779,0.000191,...,0.012821,0.046034,0.005385,0.007577,0.001410,0.001455,0.004332,0.003075,0.023331,0.000335
849,H02_20230502_080500_50,0.000732,0.000688,0.000553,0.003273,0.000911,0.000404,0.001245,0.000911,0.000231,...,0.010014,0.020023,0.003075,0.020646,0.000444,0.000710,0.001868,0.004905,0.030676,0.001648
850,H02_20230502_080500_55,0.000830,0.000392,0.000779,0.001868,0.001207,0.000296,0.002551,0.001032,0.000536,...,0.017986,0.030676,0.005730,0.025179,0.003377,0.006693,0.001701,0.004468,0.021948,0.000883


In [38]:
print("✅ Shape:", submission.shape)
print("✅ Columns:", submission.columns.tolist())
print("✅ Dtypes:\n", submission.dtypes)
print("✅ Nulls:\n", submission.isna().sum().sum())

✅ Shape: (852, 207)
✅ Columns: ['row_id', '1139490', '1192948', '1194042', '126247', '1346504', '134933', '135045', '1462711', '1462737', '1564122', '21038', '21116', '21211', '22333', '22973', '22976', '24272', '24292', '24322', '41663', '41778', '41970', '42007', '42087', '42113', '46010', '47067', '476537', '476538', '48124', '50186', '517119', '523060', '528041', '52884', '548639', '555086', '555142', '566513', '64862', '65336', '65344', '65349', '65373', '65419', '65448', '65547', '65962', '66016', '66531', '66578', '66893', '67082', '67252', '714022', '715170', '787625', '81930', '868458', '963335', 'amakin1', 'amekes', 'ampkin1', 'anhing', 'babwar', 'bafibi1', 'banana', 'baymac', 'bbwduc', 'bicwre1', 'bkcdon', 'bkmtou1', 'blbgra1', 'blbwre1', 'blcant4', 'blchaw1', 'blcjay1', 'blctit1', 'blhpar1', 'blkvul', 'bobfly1', 'bobher1', 'brtpar1', 'bubcur1', 'bubwre1', 'bucmot3', 'bugtan', 'butsal1', 'cargra1', 'cattyr', 'chbant1', 'chfmac1', 'cinbec1', 'cocher1', 'cocwoo1', 'colara1', '

In [39]:
if cfg.test_soundscapes != '/kaggle/input/birdclef-2025/test_soundscapes':
    raise RuntimeError("Test directory is not set correctly.")

RuntimeError: Test directory is not set correctly.